In [1]:
#part 0- importing libraries
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

#import the keras libraries and required packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import maxnorm

Using TensorFlow backend.


In [2]:
# part 1- importing data and cleaning it

# Importing the dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Combine the two to make data engineering easier
dataset =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)
train_len = len(train)
             
#make cabin be if cabin is known or not known
dataset['Cabin'] = pd.notnull(dataset['Cabin'])

#Fill missing embarked data with C based on https://www.kaggle.com/mrisdal/exploring-survival-on-the-titanic
dataset['Embarked'].fillna(value='C',inplace=True)

#Fill missing ticket price with average (note only the test set has missing data here)
dataset['Fare'].fillna(value=dataset['Fare'].median(), inplace = True)

#Fill blanks on age with mean age
dataset['Age'].fillna(value=round(dataset['Age'].median(),2), inplace=True)

# make a new variable of total family size
dataset['Family_size'] = dataset['Parch'] + dataset['SibSp'] + 1

# make age categories (groups, >16, >65, other?)?

# at some point fiddle with names
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
# Convert Title to categorical values
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 2 , "Mme":2, "Mlle":1, "Mrs":2, "Mr":3, "Rare":4})
dataset["Title"] = dataset["Title"].astype(int)
dataset = pd.get_dummies(dataset, columns = ["Title"],prefix="Title")

#drop name and ticket for now
dataset = dataset.drop(labels=['Ticket','Name'], axis = 1)

#order columns
dataset = dataset[['PassengerId','Sex','Embarked','Cabin','Age','Fare',  
                   'Pclass','Parch','SibSp', 'Family_size', 
                   'Title_0', 'Title_1', 'Title_2', 'Title_3', 'Title_4', 'Survived']]

# split the data back to train and test
train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=["Survived"],axis = 1,inplace=True)

#split train into x (independent) and y (dependent) variables
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

#set up the final test set
X_pred = test.iloc[:,:].values

C:\Users\James\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\James\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [3]:
# use one hot encoder to encode sex, cabin known and embarkation point on the training set
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
labelencoder_X_3 = LabelEncoder()
X[:, 3] = labelencoder_X_3.fit_transform(X[:, 3])

onehotencoder = OneHotEncoder(categorical_features = [2])
X = onehotencoder.fit_transform(X).toarray()

# use one hot encoder to encode sex, cabin known and embarkation point on the test set
labelencoder_X_4 = LabelEncoder()
X_pred[:, 1] = labelencoder_X_4.fit_transform(X_pred[:, 1])
labelencoder_X_5 = LabelEncoder()
X_pred[:, 2] = labelencoder_X_5.fit_transform(X_pred[:, 2])
labelencoder_X_6 = LabelEncoder()
X_pred[:, 3] = labelencoder_X_6.fit_transform(X_pred[:, 3])

onehotencoder = OneHotEncoder(categorical_features = [2])
X_pred = onehotencoder.fit_transform(X_pred).toarray()

Xdf= pd.DataFrame(X)
Xdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,0.0,1.0,1.0,1.0,0.0,22.0,7.2500,3.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,2.0,0.0,1.0,38.0,71.2833,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,3.0,0.0,0.0,26.0,7.9250,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,4.0,0.0,1.0,35.0,53.1000,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,5.0,1.0,0.0,35.0,8.0500,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [4]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X = sc.fit_transform(X)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_pred = sc.transform(X_pred)

In [5]:
#part 2 now let's make the ANN

#initialising the ANN
classifier = Sequential()

#adding the input layer and the first hidden layer
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu',input_dim = 17))

#adding the hidden layers
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(units=10, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dropout(0.2))

#ading the output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation= 'sigmoid'))

#compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size=10, epochs = 100 )

Epoch 1/100
712/712 [==============================] - 2s - loss: 0.6907 - acc: 0.6166     
Epoch 2/100
712/712 [==============================] - 0s - loss: 0.6837 - acc: 0.6166     
Epoch 3/100
712/712 [==============================] - 0s - loss: 0.6753 - acc: 0.6166     
Epoch 4/100
712/712 [==============================] - 0s - loss: 0.6227 - acc: 0.6166     
Epoch 5/100
712/712 [==============================] - 0s - loss: 0.5556 - acc: 0.6166     
Epoch 6/100
712/712 [==============================] - 0s - loss: 0.5380 - acc: 0.6166     
Epoch 7/100
712/712 [==============================] - 0s - loss: 0.5441 - acc: 0.7037     
Epoch 8/100
712/712 [==============================] - 0s - loss: 0.5278 - acc: 0.8188     
Epoch 9/100
712/712 [==============================] - 0s - loss: 0.5116 - acc: 0.8244     
Epoch 10/100
712/712 [==============================] - 0s - loss: 0.4986 - acc: 0.8301     
Epoch 11/100
712/712 [==============================] - 0s - loss: 0.4859 - acc

712/712 [==============================] - 0s - loss: 0.3658 - acc: 0.8610     
Epoch 89/100
712/712 [==============================] - 0s - loss: 0.3539 - acc: 0.8624     
Epoch 90/100
712/712 [==============================] - 0s - loss: 0.3665 - acc: 0.8610     
Epoch 91/100
712/712 [==============================] - 0s - loss: 0.3632 - acc: 0.8610     
Epoch 92/100
712/712 [==============================] - 0s - loss: 0.3728 - acc: 0.8596     
Epoch 93/100
712/712 [==============================] - 0s - loss: 0.3743 - acc: 0.8581     
Epoch 94/100
712/712 [==============================] - 0s - loss: 0.3648 - acc: 0.8610     
Epoch 95/100
712/712 [==============================] - 0s - loss: 0.3780 - acc: 0.8539     
Epoch 96/100
712/712 [==============================] - 0s - loss: 0.3612 - acc: 0.8567     
Epoch 97/100
712/712 [==============================] - 0s - loss: 0.3735 - acc: 0.8652     
Epoch 98/100
712/712 [==============================] - 0s - loss: 0.3798 - acc: 0.

In [6]:
#part 3 making the predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred >0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
accuracy = (cm[0,0]+cm[1,1])/cm.sum()

cm
accuracy


0.8379888268156425

In [7]:
#part 4- running the model on the real test set
y_pred = classifier.predict(X_pred)
y_pred = (y_pred >0.5)

#part 5- exporting the results for submission

results = pd.DataFrame(test['PassengerId'])
#results['Prediction']=0
results['Survived'] = y_pred
results['Survived'] = results['Survived'].astype(int)
#results
results.to_csv('Prediction3.csv', index=False)